# Team Based Model

## Imports

In [33]:

import sqlite3
import joblib
import optuna

import pandas as pd
# import plotly.graph_objects as go
# import plotly.express as px

from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold, RFE, RFECV
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.model_selection import cross_val_score

import xgboost as xgb
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import ElasticNet, LinearRegression, SGDRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pointbiserialr
from sklearn.compose import ColumnTransformer
import seaborn as sns

# import tensorflow as tf 
# from tensorflow import keras
# from tensorflow.keras import layers

from sklearn.feature_selection import SequentialFeatureSelector, RFE, RFECV
from tqdm import tqdm
from nba_api.stats.static import players, teams

import sys

%load_ext autoreload
%autoreload 2

sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
pd.options.display.max_columns=230
pd.options.display.max_rows=120

## Gather Team Level Basic, Advanced, and Scoring Data

In [35]:
def load_team_data(conn, start_season, end_season):
    """Loads basic, advanced, and scoring boxscores 
    from sqlite db and merges them into one dataframe"""
    

    basic = pd.read_sql("SELECT * FROM team_basic_boxscores", connection)
    adv = pd.read_sql("SELECT * FROM team_advanced_boxscores", connection)
    scoring = pd.read_sql("SELECT * FROM team_scoring_boxscores", connection)
    tracking = pd.read_sql("SELECT * FROM team_tracking_boxscores", connection)

    basic = basic.loc[basic['SEASON'].between(start_season, end_season)]
    basic[['GAME_ID', 'TEAM_ID']] = basic[['GAME_ID', 'TEAM_ID']].astype(str)
    adv[['GAME_ID', 'TEAM_ID']] = adv[['GAME_ID', 'TEAM_ID']].astype(str)
    scoring[['GAME_ID', 'TEAM_ID']] = scoring[['GAME_ID', 'TEAM_ID']].astype(str)
    tracking[['GAME_ID', 'TEAM_ID']] = tracking[['GAME_ID', 'TEAM_ID']].astype(str)

    df = pd.merge(basic, adv, how='left', on=[
                    'GAME_ID', 'TEAM_ID'], suffixes=['', '_y'])
    df = pd.merge(df, scoring, how='left', on=[
                  'GAME_ID', 'TEAM_ID'], suffixes=['', '_y'])
    
    df = pd.merge(df, tracking, how='left', on=['GAME_ID', 'TEAM_ID'],
                  suffixes=['', '_y'])
    

    df = df.drop(columns=['TEAM_NAME_y', 'TEAM_CITY',
                          'TEAM_ABBREVIATION_y',
                          'TEAM_CITY_y', 'MIN_y',
                          'FG_PCT_y', 'AST_y'])
    
    return df


start_season = '2013-14'
end_season = '2021-22'

connection = sqlite3.connect('../data/nba.db')
df = load_team_data(connection, start_season, end_season)
connection.close()

In [36]:
def clean_team_data(df):
    """This function cleans the team_data
    1) Changes W/L to 1/0 
    2) Changes franchise abbreviations to their most 
    recent abbreviation for consistency
    3) Converts GAME_DATE to datetime object
    4) Creates a binary column 'HOME_GAME'
    5) Removes 3 games where advanced stats were not collected
    """
    df = df.copy()
    df['WL'] = (df['WL'] == 'W').astype(int)

    abbr_mapping = {'NJN': 'BKN',
                    'CHH': 'CHA',
                    'VAN': 'MEM',
                    'NOH': 'NOP',
                    'NOK': 'NOP',
                    'SEA': 'OKC'}

    df['TEAM_ABBREVIATION'] = df['TEAM_ABBREVIATION'].replace(abbr_mapping)
    df['MATCHUP'] = df['MATCHUP'].str.replace('NJN', 'BKN')
    df['MATCHUP'] = df['MATCHUP'].str.replace('CHH', 'CHA')
    df['MATCHUP'] = df['MATCHUP'].str.replace('VAN', 'MEM')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOH', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOK', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('SEA', 'OKC')

    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

    df['HOME_GAME'] = df['MATCHUP'].str.contains('vs').astype(int)

    return df


clean_df = clean_team_data(df)
clean_df

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,DFGM,DFGA,DFG_PCT,HOME_GAME
0,2013-14,1610612746,LAC,Los Angeles Clippers,0021300003,2013-10-29,LAC @ LAL,0,240,41,83,0.494,8,21,0.381,13,23,0.565,10,30,40,27,11,4,16,21,103,-13,103.9,105.1,109.1,117.2,-5.2,-12.1,0.659,1.69,19.8,0.255,0.561,0.423,16.142,16.3,0.542,0.553,1.0,0.200,102.72,98.5,82.08,98,0.497,0.747,0.253,0.641,0.175,0.233,0.184,0.126,0.243,0.466,0.606,0.394,0.875,0.125,0.659,0.341,17.44,40,64,98,394,0,2,279,23,44,0.523,18,39,0.462,12,21,0.571,0
1,2013-14,1610612747,LAL,Los Angeles Lakers,0021300003,2013-10-29,LAL vs. LAC,1,240,42,93,0.452,14,29,0.483,18,28,0.643,18,34,52,23,8,6,19,23,116,13,109.1,117.2,103.9,105.1,5.2,12.1,0.548,1.21,15.6,0.439,0.745,0.577,17.871,19.2,0.527,0.551,1.0,0.195,102.72,98.5,82.08,99,0.503,0.688,0.312,0.483,0.121,0.362,0.112,0.155,0.241,0.362,0.429,0.571,0.786,0.214,0.548,0.452,17.35,44,64,103,449,4,0,318,15,31,0.484,27,62,0.435,20,28,0.714,1
2,2013-14,1610612741,CHI,Chicago Bulls,0021300002,2013-10-29,CHI @ MIA,0,240,35,83,0.422,7,26,0.269,18,23,0.783,11,30,41,23,11,4,19,27,95,-12,93.9,96.9,107.3,108.1,-13.3,-11.1,0.657,1.21,17.0,0.294,NaN,NaN,18.790,19.4,0.464,0.510,1.0,0.198,100.44,98.5,82.08,98,0.401,0.687,0.313,0.589,0.063,0.221,0.179,0.189,0.316,0.526,0.571,0.429,1.000,0.000,0.657,0.343,17.64,37,57,87,446,3,3,317,17,35,0.486,18,48,0.375,11,20,0.550,0
3,2013-14,1610612748,MIA,Miami Heat,0021300002,2013-10-29,MIA vs. CHI,1,240,37,72,0.514,11,20,0.550,22,29,0.759,5,35,40,26,10,7,20,21,107,12,107.3,108.1,93.9,96.9,13.3,11.1,0.703,1.30,19.9,NaN,0.706,NaN,20.048,20.2,0.590,0.631,1.0,0.197,100.44,98.5,82.08,99,0.599,0.722,0.278,0.486,0.168,0.308,0.093,0.206,0.140,0.318,0.577,0.423,1.000,0.000,0.703,0.297,17.01,22,73,93,462,3,3,340,11,28,0.393,26,44,0.591,17,26,0.654,1
4,2013-14,1610612754,IND,Indiana Pacers,0021300001,2013-10-29,IND vs. ORL,1,240,34,71,0.479,7,17,0.412,22,32,0.688,10,34,44,17,4,18,21,13,97,10,101.0,103.2,84.1,92.6,16.8,10.6,0.500,0.81,13.8,0.317,0.617,0.495,21.857,22.3,0.528,0.570,1.0,0.198,99.74,94.0,78.33,94,0.661,0.761,0.239,0.557,0.186,0.216,0.134,0.227,0.299,0.371,0.444,0.556,0.714,0.286,0.500,0.500,16.09,40,78,117,428,1,1,310,20,38,0.526,14,33,0.424,9,21,0.429,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23001,2021-22,1610612744,GSW,Golden State Warriors,0042100404,2022-06-10,GSW @ BOS,1,240,40,91,0.440,15,43,0.349,12,15,0.800,16,39,55,20,10,5,16,21,107,10,109.6,111.5,98.6,101.0,11.0,10.4,0.500,1.25,15.0,0.385,0.736,0.562,16.393,16.7,0.522,0.548,1.0,0.201,97.98,96.0,80.00,96,0.552,0.527,0.473,0.467,0.112,0.421,0.075,0.112,0.178,0.355,0.400,0.600,0.667,0.333,0.500,0.500,17.19,28,60,85,373,3,2,247,20,38,0.526,20,53,0.377,12,18,0.667,0
23002,2021-22,1610612738,BOS,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,0,240,31,75,0.413,11,32,0.344,21,31,0.677,8,39,47,18,2,2,18,16,94,-10,95.3,95.9,106.6,107.2,-11.3,-11.3,0.581,1.00,14.4,0.250,0.894,0.568,18.248,18.4,0.487,0.530,1.0,0.201,98.12,97.5,81.25,98,0.438,0.573,0.427,0.426,0.043,0.351,0.085,0.223,0.096,0.383,0.500,0.500,0.727,0.273,0.581,0.419,17.06,17,58,75,397,1,2,278,12,27,0.444,19,48,

In [37]:
# from src.data.make_team_dataset import prep_for_aggregation

def prep_for_aggregation(df):
    """This function...
    1) Removes categories that are percentages,
    as we will be averaging them and do not want to average 
    percentages. 
    2) Converts shooting percentage stats into raw values"""
    df = df.copy()

    df = df.drop(columns=['FT_PCT', 'FG_PCT', 'FG3_PCT', 'DREB_PCT',
                          'OREB_PCT', 'REB_PCT', 'AST_PCT', 'AST_TOV',
                          'AST_RATIO', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
                          'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT',
                          'PACE_PER40', 'MIN', 'PIE', 'CFG_PCT', 'UFG_PCT',
                          'DFG_PCT', 'E_OFF_RATING', 'E_DEF_RATING', 'E_NET_RATING'])

    df['FG2M'] = df['FGM'] - df['FG3M']
    df['FG2A'] = df['FGA'] - df['FG3A']
    df['PTS_2PT_MR'] = (df['PTS'] * df['PCT_PTS_2PT_MR']).astype('int8')
    df['PTS_FB'] = (df['PTS'] * df['PCT_PTS_FB']).astype('int8')
    df['PTS_OFF_TOV'] = (df['PTS'] * df['PCT_PTS_OFF_TOV']).astype('int8')
    df['PTS_PAINT'] = (df['PTS'] * df['PCT_PTS_PAINT']).astype('int8')
    df['AST_2PM'] = (df['FG2M'] * df['PCT_AST_2PM']).astype('int8')
    df['AST_3PM'] = (df['FG3M'] * df['PCT_AST_3PM']).astype('int8')
    df['UAST_2PM'] = (df['FG2M'] * df['PCT_UAST_2PM']).astype('int8')
    df['UAST_3PM'] = (df['FG3M'] * df['PCT_UAST_3PM']).astype('int8')

    df['POINT_DIFF'] = df['PLUS_MINUS']
    df['RECORD'] = df['WL']
    df['TEAM_SCORE'] = df['PTS']
    
    df = df.drop(columns = ['PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_2PT',
                          'PCT_PTS_2PT_MR', 'PCT_PTS_3PT', 'PCT_PTS_FB',
                          'PCT_PTS_FT','PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT', 
                          'PCT_AST_2PM', 'PCT_UAST_2PM','PCT_AST_3PM',
                          'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM',
                          'E_PACE'])
    
    ## Reorder Columns
    
    df = df[['SEASON', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'TEAM_SCORE', 'WL', 'POINT_DIFF', 'HOME_GAME', 'RECORD',
       'FG2M', 'FG2A', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'PACE',
       'POSS', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS',
       'CFGM', 'CFGA', 'UFGM', 'UFGA', 'DFGM', 'DFGA', 'PTS_2PT_MR', 'PTS_FB', 'PTS_OFF_TOV', 'PTS_PAINT', 'AST_2PM',
       'AST_3PM', 'UAST_2PM', 'UAST_3PM']]

    return df


clean_df2 = prep_for_aggregation(clean_df)

clean_df2

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,TEAM_SCORE,WL,POINT_DIFF,HOME_GAME,RECORD,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,OFF_RATING,DEF_RATING,NET_RATING,PACE,POSS,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM
0,2013-14,1610612746,LAC,Los Angeles Clippers,0021300003,2013-10-29,LAC @ LAL,103,0,-13,0,0,33,62,8,21,13,23,10,30,40,27,11,4,16,21,103,-13,105.1,117.2,-12.1,98.5,98,17.44,40,64,98,394,0,2,279,23,44,18,39,12,21,18,18,25,47,19,7,13,1
1,2013-14,1610612747,LAL,Los Angeles Lakers,0021300003,2013-10-29,LAL vs. LAC,116,1,13,1,1,28,64,14,29,18,28,18,34,52,23,8,6,19,23,116,13,117.2,105.1,12.1,98.5,99,17.35,44,64,103,449,4,0,318,15,31,27,62,20,28,14,12,27,41,12,11,15,2
2,2013-14,1610612741,CHI,Chicago Bulls,0021300002,2013-10-29,CHI @ MIA,95,0,-12,0,0,28,57,7,26,18,23,11,30,41,23,11,4,19,27,95,-12,96.9,108.1,-11.1,98.5,98,17.64,37,57,87,446,3,3,317,17,35,18,48,11,20,5,17,30,49,15,7,12,0
3,2013-14,1610612748,MIA,Miami Heat,0021300002,2013-10-29,MIA vs. CHI,107,1,12,1,1,26,52,11,20,22,29,5,35,40,26,10,7,20,21,107,12,108.1,96.9,11.1,98.5,99,17.01,22,73,93,462,3,3,340,11,28,26,44,17,26,17,9,14,34,15,11,10,0
4,2013-14,1610612754,IND,Indiana Pacers,0021300001,2013-10-29,IND vs. ORL,97,1,10,1,1,27,54,7,17,22,32,10,34,44,17,4,18,21,13,97,10,103.2,92.6,10.6,94.0,94,16.09,40,78,117,428,1,1,310,20,38,14,33,9,21,18,12,29,35,11,4,15,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23001,2021-22,1610612744,GSW,Golden State Warriors,0042100404,2022-06-10,GSW @ BOS,107,1,10,0,1,25,48,15,43,12,15,16,39,55,20,10,5,16,21,107,10,111.5,101.0,10.4,96.0,96,17.19,28,60,85,373,3,2,247,20,38,20,53,12,18,11,8,19,37,10,10,15,4
23002,2021-22,1610612738,BOS,Boston Celtics,0042100405,2022-06-13,BOS @ GSW,94,0,-10,0,0,20,43,11,32,21,31,8,39,47,18,2,2,18,16,94,-10,95.9,107.2,-11.3,97.5,98,17.06,17,58,75,397,1,2,278,12,27,19,48,14,22,4,7,9,36,10,7,10,3
23003,2021-22,1610612744,GSW,Golden State Warriors,0042100405,2022-06-13,GSW vs. BOS,104,1,10,1,1,32,48,9,40,13,15,4,35,39,23,9,2,7,28,104,10,107.2,95.9,11.3,97.5,97,17.59,20,52,68,379,1,2,265,19,30,22,58,14,24,14,11,22,50,16,7,16,1
23004,2021-22,1610612738,BOS,Boston Celtics,0042100406,2022-06-16,BOS vs. GSW,90,0,-13,1,0,23,52,11,28,11,12,11,30,41,27,8,8,23,16,90,-13,96.8,109.6,-12.8,93.5,93,16.48,33,47,80,400,5,2,276,15,40,19,40,13,25,8,11,27,37,16,9,6,1


## Add Betting Data 

In [38]:
def load_betting_data(conn):
    spreads = pd.read_sql("SELECT * FROM spreads", conn)
    moneylines = pd.read_sql("SELECT * FROM moneylines", conn)

    return spreads, moneylines


connection = sqlite3.connect("../data/nba.db")
spreads, moneylines = load_betting_data(connection)
connection.close()

In [39]:
def convert_american_to_decimal(x):
    return np.where(x>0, (100+x)/100, 1+(100.0/-x))          
          
                

In [40]:
def clean_moneyline_df(df=moneylines):
    abbr_mapping = {'Boston': 'BOS', 'Portland': 'POR',
                    'L.A. Lakers': 'LAL', 'Brooklyn': 'BKN',
                    'Cleveland': 'CLE', 'Toronto': 'TOR',
                    'Philadelphia': 'PHI', 'Memphis': 'MEM',
                    'Minnesota': 'MIN', 'New Orleans': 'NOP',
                    'Oklahoma City': 'OKC', 'Dallas': 'DAL',
                    'San Antonio': 'SAS', 'Denver': 'DEN',
                    'Golden State': 'GSW', 'L.A. Clippers': 'LAC',
                    'Orlando': 'ORL', 'Utah': 'UTA',
                    'Charlotte': 'CHA', 'Detroit': 'DET',
                    'Miami': 'MIA', 'Phoenix': 'PHX',
                    'Atlanta': 'ATL', 'New York': 'NYK',
                    'Indiana': 'IND', 'Chicago': 'CHI',
                    'Houston': 'HOU', 'Milwaukee': 'MIL',
                    'Sacramento': 'SAC', 'Washington': 'WAS'}

    df['HOME_TEAM'] = df['HOME_TEAM'].replace(abbr_mapping)
    df['AWAY_TEAM'] = df['AWAY_TEAM'].replace(abbr_mapping)

    away_mls = df['AWAY_ML'].str.split(",", expand=True)
    home_mls = df['HOME_ML'].str.split(",", expand=True)

    away_mls = away_mls.replace('-', np.nan).replace('', np.nan)
    away_mls = away_mls.fillna(value=np.nan)
    away_mls = away_mls.astype(float)

    home_mls = home_mls.replace('-', np.nan).replace('', np.nan)
    home_mls = home_mls.fillna(value=np.nan)
    home_mls = home_mls.astype(float)

    highest_away_ml = away_mls.apply(lambda row: np.nanmax(
        abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
    highest_away_ml = convert_american_to_decimal(highest_away_ml)
    highest_away_ml = pd.DataFrame(
        highest_away_ml, columns=['HIGHEST_AWAY_ML'])

    highest_home_ml = home_mls.apply(lambda row: np.nanmax(
        abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
    highest_home_ml = convert_american_to_decimal(highest_home_ml)
    highest_home_ml = pd.DataFrame(
        highest_home_ml, columns=['HIGHEST_HOME_ML'])

    moneylines = pd.concat(
        [df.iloc[:, :4], highest_home_ml, highest_away_ml], axis=1)
    
    moneylines['GM_DATE'] = pd.to_datetime(moneylines['GM_DATE'])

    return moneylines

In [41]:
clean_mls = clean_moneyline_df(df=moneylines)

C:\Users\Jordan Nishimura\AppData\Local\Temp\ipykernel_38028\4006584993.py:33: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
C:\Users\Jordan Nishimura\AppData\Local\Temp\ipykernel_38028\4006584993.py:39: RuntimeWarning: All-NaN axis encountered
  abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)


In [42]:
def clean_spreads_df(df=spreads):
    abbr_mapping = {'Boston': 'BOS', 'Portland': 'POR',
                    'L.A. Lakers': 'LAL', 'Brooklyn': 'BKN',
                    'Cleveland': 'CLE', 'Toronto': 'TOR',
                    'Philadelphia': 'PHI', 'Memphis': 'MEM',
                    'Minnesota': 'MIN', 'New Orleans': 'NOP',
                    'Oklahoma City': 'OKC', 'Dallas': 'DAL',
                    'San Antonio': 'SAS', 'Denver': 'DEN',
                    'Golden State': 'GSW', 'L.A. Clippers': 'LAC',
                    'Orlando': 'ORL', 'Utah': 'UTA',
                    'Charlotte': 'CHA', 'Detroit': 'DET',
                    'Miami': 'MIA', 'Phoenix': 'PHX',
                    'Atlanta': 'ATL', 'New York': 'NYK',
                    'Indiana': 'IND', 'Chicago': 'CHI',
                    'Houston': 'HOU', 'Milwaukee': 'MIL',
                    'Sacramento': 'SAC', 'Washington': 'WAS'}

    df['HOME_TEAM'] = df['HOME_TEAM'].replace(abbr_mapping)
    df['AWAY_TEAM'] = df['AWAY_TEAM'].replace(abbr_mapping)

    away_spreads = df['AWAY_SPREAD'].str.split(",", expand=True)
    home_spreads = df['HOME_SPREAD'].str.split(",", expand=True)

    for col in away_spreads.columns:
        away_spreads[col] = away_spreads[col].str[:-4]
        away_spreads[col] = away_spreads[col].str.replace('½', '.5')
        away_spreads[col] = away_spreads[col].str.replace('PK', '0')

        away_spreads[col] = away_spreads[col].astype(str).apply(
            lambda x: x if x == '' else (x[:-1] if x[-1] == '-' else x))

    away_spreads = away_spreads.replace('-', np.nan)
    away_spreads = away_spreads.replace('', np.nan)
    away_spreads = away_spreads.replace('None', np.nan)
    away_spreads = away_spreads.fillna(value=np.nan)

    away_spreads = away_spreads.astype(float)

    for col in home_spreads.columns:
        home_spreads[col] = home_spreads[col].str[:-4]
        home_spreads[col] = home_spreads[col].str.replace('½', '.5')
        home_spreads[col] = home_spreads[col].str.replace('PK', '0')

        home_spreads[col] = home_spreads[col].astype(str).apply(
            lambda x: x if x == '' else (x[:-1] if x[-1] == '-' else x))

    home_spreads = home_spreads.replace('-', np.nan).replace('', np.nan).replace('None', np.nan)
    home_spreads = home_spreads.fillna(value=np.nan)

    home_spreads = home_spreads.astype(float)

    highest_away_spread = away_spreads.apply(
        lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
    highest_away_spread = pd.DataFrame(
        highest_away_spread, columns=['HIGHEST_AWAY_SPREAD'])

    highest_home_spread = home_spreads.apply(
        lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
    highest_home_spread = pd.DataFrame(
        highest_home_spread, columns=['HIGHEST_HOME_SPREAD'])

    spreads = pd.concat(
        [df.iloc[:, :4], highest_home_spread, highest_away_spread], axis=1)
    spreads['GM_DATE'] = pd.to_datetime(spreads['GM_DATE'])

    return spreads

In [43]:
clean_spreads = clean_spreads_df(df = spreads)

C:\Users\Jordan Nishimura\AppData\Local\Temp\ipykernel_38028\1082226645.py:53: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
C:\Users\Jordan Nishimura\AppData\Local\Temp\ipykernel_38028\1082226645.py:58: RuntimeWarning: All-NaN axis encountered
  lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)


In [44]:
clean_spreads

,SEASON,GM_DATE,HOME_TEAM,AWAY_TEAM,HIGHEST_HOME_SPREAD,HIGHEST_AWAY_SPREAD
0,2006-07,2006-10-31,MIA,CHI,-4.5,4.5
1,2006-07,2006-10-31,LAL,PHX,8.5,-8.5
2,2006-07,2006-11-02,DAL,SAS,-3.0,3.0
3,2006-07,2006-11-02,LAC,DEN,-4.5,4.5
4,2006-07,2006-11-03,MEM,CHA,-5.0,5.0
...,...,...,...,...,...,...
20197,2021-22,2022-06-05,GSW,BOS,-5.5,5.0
20198,2021-22,2022-06-08,BOS,GSW,-3.5,3.5
20199,2021-22,2022-06-10,BOS,GSW,-4.0,4.0
20200,2021-22,2022-06-13,GSW,BOS,-4.0,4.0


In [45]:
clean_mls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20200 entries, 0 to 20199
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   SEASON           20200 non-null  object        
 1   GM_DATE          20200 non-null  datetime64[ns]
 2   HOME_TEAM        20200 non-null  object        
 3   AWAY_TEAM        20200 non-null  object        
 4   HIGHEST_HOME_ML  18507 non-null  float64       
 5   HIGHEST_AWAY_ML  18506 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 947.0+ KB


In [46]:
def merge_betting_and_boxscore_data(clean_spreads, clean_mls, clean_boxscores):
    clean_boxscores['HOME_TEAM'] = clean_boxscores['MATCHUP'].apply(
        lambda x: x[:3] if 'vs' in x else x[-3:])
    clean_boxscores['AWAY_TEAM'] = clean_boxscores['MATCHUP'].apply(
        lambda x: x[:3] if '@' in x else x[-3:])

    temp = pd.merge(clean_mls, clean_spreads, on=[
                    'SEASON', 'GM_DATE', 'HOME_TEAM', 'AWAY_TEAM'])

    merged_df = pd.merge(clean_boxscores, temp, how='left', 
                         left_on=['SEASON', 'HOME_TEAM', 'AWAY_TEAM', 'GAME_DATE'],
                         right_on=['SEASON', 'HOME_TEAM', 'AWAY_TEAM', 'GM_DATE'])

    merged_df['ML'] = merged_df.apply(lambda row: row['HIGHEST_HOME_ML'] if row['HOME_GAME'] == 1
                                      else row['HIGHEST_AWAY_ML'], axis=1)

    merged_df['SPREAD'] = merged_df.apply(lambda row: row['HIGHEST_HOME_SPREAD'] if row['HOME_GAME'] == 1
                                          else -row['HIGHEST_HOME_SPREAD'], axis=1)

    merged_df = merged_df.drop(columns=['HOME_TEAM', 'AWAY_TEAM', 'GM_DATE',
                                        'HIGHEST_HOME_ML', 'HIGHEST_AWAY_ML',
                                        'HIGHEST_HOME_SPREAD', 'HIGHEST_AWAY_SPREAD'])

    merged_df['ATS_DIFF'] = merged_df['POINT_DIFF'] + merged_df['SPREAD']

    merged_df['TEAM_COVERED'] = (merged_df['ATS_DIFF'] > 0).astype(int)
    
#     merged_df.loc[merged_df['ATS_DIFF'] == 0.0, 'TEAM_COVERED'] = 0.5

    return merged_df


merged_df = merge_betting_and_boxscore_data(
    clean_spreads, clean_mls, clean_boxscores=clean_df2)

In [80]:
merged_df[['TEAM_ABBREVIATION', 'GAME_DATE', 'HOME_GAME', 'SPREAD']]

,TEAM_ABBREVIATION,GAME_DATE,HOME_GAME,SPREAD
0,LAC,2013-10-29,0,-10.0
1,LAL,2013-10-29,1,10.0
2,CHI,2013-10-29,0,5.0
3,MIA,2013-10-29,1,-5.0
4,IND,2013-10-29,1,-12.5
...,...,...,...,...
23001,GSW,2022-06-10,0,4.0
23002,BOS,2022-06-13,0,4.0
23003,GSW,2022-06-13,1,-4.0
23004,BOS,2022-06-16,1,-4.0


In [47]:
def normalize_per_100_poss(df):
    df = df.copy(deep=True)
    
    df.iloc[:, 12:27] = 100*df.iloc[:, 12:27].div(df['PACE'], axis=0) 
    df.iloc[:,  34:-4] = 100*df.iloc[:, 34:-4].div(df['PACE'], axis=0) 
    
    return df

stats_per_100 = normalize_per_100_poss(merged_df)
stats_per_100.head()

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,TEAM_SCORE,WL,POINT_DIFF,HOME_GAME,RECORD,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,OFF_RATING,DEF_RATING,NET_RATING,PACE,POSS,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,ML,SPREAD,ATS_DIFF,TEAM_COVERED
0,2013-14,1610612746,LAC,Los Angeles Clippers,0021300003,2013-10-29,LAC @ LAL,103,0,-13,0,0,33.502538,62.944162,8.121827,21.319797,13.197970,23.350254,10.152284,30.456853,40.609137,27.411168,11.167513,4.060914,16.243655,21.319797,104.568528,-13,105.1,117.2,-12.1,98.5,98,17.44,40.609137,64.974619,99.492386,400.000000,0.000000,2.030457,283.248731,23.350254,44.670051,18.274112,39.593909,12.182741,21.319797,18.274112,18.274112,25.380711,47.715736,19.289340,7.106599,13.197970,1.015228,1.156250,-10.0,-23.0,0
1,2013-14,1610612747,LAL,Los Angeles Lakers,0021300003,2013-10-29,LAL vs. LAC,116,1,13,1,1,28.426396,64.974619,14.213198,29.441624,18.274112,28.426396,18.274112,34.517766,52.791878,23.350254,8.121827,6.091371,19.289340,23.350254,117.766497,13,117.2,105.1,12.1,98.5,99,17.35,44.670051,64.974619,104.568528,455.837563,4.060914,0.000000,322.842640,15.228426,31.472081,27.411168,62.944162,20.304569,28.426396,14.213198,12.182741,27.411168,41.624365,12.182741,11.167513,15.228426,2.030457,5.550000,10.0,23.0,1
2,2013-14,1610612741,CHI,Chicago Bulls,0021300002,2013-10-29,CHI @ MIA,95,0,-12,0,0,28.426396,57.868020,7.106599,26.395939,18.274112,23.350254,11.167513,30.456853,41.624365,23.350254,11.167513,4.060914,19.289340,27.411168,96.446701,-12,96.9,108.1,-11.1,98.5,98,17.64,37.563452,57.868020,88.324873,452.791878,3.045685,3.045685,321.827411,17.258883,35.532995,18.274112,48.730964,11.167513,20.304569,5.076142,17.258883,30.456853,49.746193,15.228426,7.106599,12.182741,0.000000,2.850000,5.0,-7.0,0
3,2013-14,1610612748,MIA,Miami Heat,0021300002,2013-10-29,MIA vs. CHI,107,1,12,1,1,26.395939,52.791878,11.167513,20.304569,22.335025,29.441624,5.076142,35.532995,40.609137,26.395939,10.152284,7.106599,20.304569,21.319797,108.629442,12,108.1,96.9,11.1,98.5,99,17.01,22.335025,74.111675,94.416244,469.035533,3.045685,3.045685,345.177665,11.167513,28.426396,26.395939,44.670051,17.258883,26.395939,17.258883,9.137056,14.213198,34.517766,15.228426,11.167513,10.152284,0.000000,1.454545,-5.0,7.0,1
4,2013-14,1610612754,IND,Indiana Pacers,0021300001,2013-10-29,IND vs. ORL,97,1,10,1,1,28.723404,57.446809,7.446809,18.085106,23.404255,34.042553,10.638298,36.170213,46.808511,18.085106,4.255319,19.148936,22.340426,13.829787,103.191489,10,103.2,92.6,10.6,94.0,94,16.09,42.553191,82.978723,124.468085,455.319149,1.063830,1.063830,329.787234,21.276596,40.425532,14.893617,35.106383,9.574468,22.340426,19.148936,12.765957,30.851064,37.234043,11.702128,4.255319,15.957447,2.127660,1.090498,-12.5,-2.5,0


In [48]:
stats_per_100.loc[(stats_per_100['GAME_DATE'] == '2021-10-20')
                  &(stats_per_100['TEAM_ABBREVIATION'] == 'BOS')]

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,TEAM_SCORE,WL,POINT_DIFF,HOME_GAME,RECORD,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,OFF_RATING,DEF_RATING,NET_RATING,PACE,POSS,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,ML,SPREAD,ATS_DIFF,TEAM_COVERED
20389,2021-22,1610612738,BOS,Boston Celtics,0022100005,2021-10-20,BOS @ NYK,134,0,-4,0,0,26.205959,58.235465,20.382413,55.323692,16.500049,22.323595,14.558866,39.794235,54.353101,33.000097,12.617684,8.73532,17.47064,23.294186,130.059206,-4,108.1,110.4,-2.3,103.03,124,20.49,39.794235,53.38251,89.29438,469.766088,6.794138,2.911773,317.383286,11.647093,29.117733,34.941279,84.441425,22.323595,31.058915,11.647093,18.441231,16.500049,39.794235,15.529457,16.500049,9.705911,2.911773,2.2,3.5,-0.5,0


In [96]:
def create_matchups(df):
    """This function makes each row a matchup between 
    team and opp"""
    df = df.copy()
    

    matchups = pd.merge(df, df.iloc[:, :-4], on=['GAME_ID'], suffixes=['', '_opp'])
    matchups = matchups.loc[matchups['TEAM_ABBREVIATION'] != matchups['TEAM_ABBREVIATION_opp']]

    matchups = matchups.drop(columns = ['SEASON_opp',
                             'TEAM_ABBREVIATION_opp', 'GAME_DATE_opp',
                             'MATCHUP_opp', 'HOME_GAME_opp', 'TEAM_NAME_opp',
                                       'TEAM_ID_opp', 'WL_opp', ]
                 )
    
    matchups
    
    return matchups


matchups = create_matchups(stats_per_100)
matchups.head()

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,TEAM_SCORE,WL,POINT_DIFF,HOME_GAME,RECORD,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,OFF_RATING,DEF_RATING,NET_RATING,PACE,POSS,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,ML,SPREAD,ATS_DIFF,TEAM_COVERED,TEAM_SCORE_opp,POINT_DIFF_opp,RECORD_opp,FG2M_opp,FG2A_opp,FG3M_opp,FG3A_opp,FTM_opp,FTA_opp,OREB_opp,DREB_opp,REB_opp,AST_opp,STL_opp,BLK_opp,TOV_opp,PF_opp,PTS_opp,PLUS_MINUS_opp,OFF_RATING_opp,DEF_RATING_opp,NET_RATING_opp,PACE_opp,POSS_opp,DIST_opp,ORBC_opp,DRBC_opp,RBC_opp,TCHS_opp,SAST_opp,FTAST_opp,PASS_opp,CFGM_opp,CFGA_opp,UFGM_opp,UFGA_opp,DFGM_opp,DFGA_opp,PTS_2PT_MR_opp,PTS_FB_opp,PTS_OFF_TOV_opp,PTS_PAINT_opp,AST_2PM_opp,AST_3PM_opp,UAST_2PM_opp,UAST_3PM_opp
1,2013-14,1610612746,LAC,Los Angeles Clippers,0021300003,2013-10-29,LAC @ LAL,103,0,-13,0,0,33.502538,62.944162,8.121827,21.319797,13.197970,23.350254,10.152284,30.456853,40.609137,27.411168,11.167513,4.060914,16.243655,21.319797,104.568528,-13,105.1,117.2,-12.1,98.5,98,17.44,40.609137,64.974619,99.492386,400.000000,0.000000,2.030457,283.248731,23.350254,44.670051,18.274112,39.593909,12.182741,21.319797,18.274112,18.274112,25.380711,47.715736,19.289340,7.106599,13.197970,1.015228,1.156250,-10.0,-23.0,0,116,13,1,28.426396,64.974619,14.213198,29.441624,18.274112,28.426396,18.274112,34.517766,52.791878,23.350254,8.121827,6.091371,19.289340,23.350254,117.766497,13,117.2,105.1,12.1,98.5,99,17.35,44.670051,64.974619,104.568528,455.837563,4.060914,0.000000,322.842640,15.228426,31.472081,27.411168,62.944162,20.304569,28.426396,14.213198,12.182741,27.411168,41.624365,12.182741,11.167513,15.228426,2.030457
2,2013-14,1610612747,LAL,Los Angeles Lakers,0021300003,2013-10-29,LAL vs. LAC,116,1,13,1,1,28.426396,64.974619,14.213198,29.441624,18.274112,28.426396,18.274112,34.517766,52.791878,23.350254,8.121827,6.091371,19.289340,23.350254,117.766497,13,117.2,105.1,12.1,98.5,99,17.35,44.670051,64.974619,104.568528,455.837563,4.060914,0.000000,322.842640,15.228426,31.472081,27.411168,62.944162,20.304569,28.426396,14.213198,12.182741,27.411168,41.624365,12.182741,11.167513,15.228426,2.030457,5.550000,10.0,23.0,1,103,-13,0,33.502538,62.944162,8.121827,21.319797,13.197970,23.350254,10.152284,30.456853,40.609137,27.411168,11.167513,4.060914,16.243655,21.319797,104.568528,-13,105.1,117.2,-12.1,98.5,98,17.44,40.609137,64.974619,99.492386,400.000000,0.000000,2.030457,283.248731,23.350254,44.670051,18.274112,39.593909,12.182741,21.319797,18.274112,18.274112,25.380711,47.715736,19.289340,7.106599,13.197970,1.015228
5,2013-14,1610612741,CHI,Chicago Bulls,0021300002,2013-10-29,CHI @ MIA,95,0,-12,0,0,28.426396,57.868020,7.106599,26.395939,18.274112,23.350254,11.167513,30.456853,41.624365,23.350254,11.167513,4.060914,19.289340,27.411168,96.446701,-12,96.9,108.1,-11.1,98.5,98,17.64,37.563452,57.868020,88.324873,452.791878,3.045685,3.045685,321.827411,17.258883,35.532995,18.274112,48.730964,11.167513,20.304569,5.076142,17.258883,30.456853,49.746193,15.228426,7.106599,12.182741,0.000000,2.850000,5.0,-7.0,0,107,12,1,26.395939,52.791878,11.167513,20.304569,22.335025,29.441624,5.076142,35.532995,40.609137,26.395939,10.152284,7.106599,20.304569,21.319797,108.629442,12,108.1,96.9,11.1,98.5,99,17.01,22.335025,74.111675,94.416244,469.035533,3.045685,3.045685,345.177665,11.167513,28.426396,26.395939,44.670051,17.258883,26.395939,17.258883,9.137056,14.213198,34.517766,15.228426,11.167513,10.152284,0.000000
6,2013-14,1610612748,MIA,Miami Heat,0021300002,2013-10-29,MIA vs. CHI,107,1,12,1,1,26.395939,52.791878,11.167513,20.304569,22.335025,29.441624,5.076142,35.532995,40.609137,26.395939,10.152284,7.106599,20.304569,21.319797,108.629442,12,108.1,96.9,11.1,98.5,99,17.01,22.335025,74.111675,94.416244,469.035533,3.045685,3.045685,345.177665,11.167513,28.426396,26.395939,44.670051,17.258883,26.395939,17.258883,9

In [100]:
matchups[['TEAM_ABBREVIATION', 'MATCHUP', 'TEAM_SCORE', 'POINT_DIFF', 'SPREAD', 'ATS_DIFF']]

,TEAM_ABBREVIATION,MATCHUP,TEAM_SCORE,POINT_DIFF,SPREAD,ATS_DIFF
1,LAC,LAC @ LAL,103,-13,-10.0,-23.0
2,LAL,LAL vs. LAC,116,13,10.0,23.0
5,CHI,CHI @ MIA,95,-12,5.0,-7.0
6,MIA,MIA vs. CHI,107,12,-5.0,7.0
9,IND,IND vs. ORL,97,10,-12.5,-2.5
...,...,...,...,...,...,...
46002,GSW,GSW @ BOS,107,10,4.0,14.0
46005,BOS,BOS @ GSW,94,-10,4.0,-6.0
46006,GSW,GSW vs. BOS,104,10,-4.0,6.0
46009,BOS,BOS vs. GSW,90,-13,-4.0,-17.0


In [98]:
def build_team_avg_stats_df(df: pd.DataFrame, span = 10) -> pd.DataFrame:    
    """This function finds the average for each team and opp statistic up to (and NOT including) the given date.
    """
    
    df = df.copy(deep=True)

    df = df.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE']).reset_index(drop=True)

    

    drop_cols = ['TEAM_ID', 'TEAM_NAME', 'GAME_ID', 'MATCHUP', 
                 'HOME_GAME', 'TEAM_SCORE', 'ML', 'SPREAD', 
                'GAME_DATE', 'POINT_DIFF', 'WL', 'TEAM_SCORE_opp',
                'POINT_DIFF_opp', 'RECORD', 'RECORD_opp', 'TEAM_COVERED']

    stats = df.drop(columns=drop_cols)

    avg_stat_holder = []

    for stat in stats.columns[2:]:
        avg_stats = stats.groupby(['TEAM_ABBREVIATION'])[stat].ewm(span=span).mean().reset_index(drop=True)
        avg_stat_holder.append(avg_stats)
    
    
    matchup_info = df[['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                          'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                          'ML', 'SPREAD', 'ATS_DIFF', 'RECORD', 'TEAM_COVERED', 
                          'POINT_DIFF', 'WL']]   

    avg_stats = pd.concat(avg_stat_holder, axis=1)
    

    avg_stats = avg_stats.rename(columns={'ATS_DIFF':'AVG_ATS_DIFF'})
    
    avg_stats = pd.concat([matchup_info, avg_stats], axis=1)
    
    avg_stats['WIN_PCT'] = avg_stats.groupby(['TEAM_ABBREVIATION'])['RECORD'].rolling(window=span).mean().values
    avg_stats['COVER_PCT'] = avg_stats.groupby(['TEAM_ABBREVIATION'])['TEAM_COVERED'].rolling(window=span).mean().values

    avg_stats = avg_stats.drop(columns='RECORD')

    avg_stats = avg_stats.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE'])
    avg_stats.iloc[:, 14:] = avg_stats.iloc[:, 14:].shift(1).where(avg_stats['TEAM_ABBREVIATION'].eq(avg_stats['TEAM_ABBREVIATION'].shift()))

    avg_stats = avg_stats.add_suffix('_L{}'.format(span))
    
    avg_stats = avg_stats.rename(columns = {'SEASON_L{}'.format(span):'SEASON',
                                           'TEAM_ABBREVIATION_L{}'.format(span):'TEAM_ABBREVIATION',
                                           'GAME_DATE_L{}'.format(span):'GAME_DATE',
                                           'GAME_ID_L{}'.format(span):'GAME_ID',
                                           'MATCHUP_L{}'.format(span): 'MATCHUP', 
                                           'HOME_GAME_L{}'.format(span): 'HOME_GAME', 
                                           'TEAM_SCORE_L{}'.format(span):'TEAM_SCORE',
                                           'ML_L{}'.format(span):'ML', 
                                           'SPREAD_L{}'.format(span):'SPREAD',
                                           'ATS_DIFF_L{}'.format(span):'ATS_DIFF',
                                           'RECORD_L{}'.format(span):'RECORD', 
                                           'TEAM_COVERED_L{}'.format(span):'TEAM_COVERED',
                                           'POINT_DIFF_L{}'.format(span):'POINT_DIFF',
                                           'WL_L{}'.format(span):'WL'})
    
    return avg_stats

team_stats_ewa_5 = build_team_avg_stats_df(matchups, span=5)




In [51]:
team_stats_ewa_5.loc[team_stats_ewa_5['TEAM_ABBREVIATION'] == 'GSW', ['MATCHUP', 'SPREAD', 'POINT_DIFF', 'ATS_DIFF', 'TEAM_SCORE', 'AVG_ATS_DIFF_L5']]

,MATCHUP,SPREAD,POINT_DIFF,ATS_DIFF,TEAM_SCORE,AVG_ATS_DIFF_L5
6865,GSW vs. LAL,12.0,31,-19.0,125,NaN
6866,GSW @ LAC,-6.0,-11,5.0,115,-19.000000
6867,GSW vs. SAC,11.0,11,0.0,98,-4.600000
6868,GSW @ PHI,NaN,20,NaN,110,-2.421053
6869,GSW @ MIN,1.0,13,-12.0,106,-2.421053
...,...,...,...,...,...,...
7705,GSW vs. BOS,5.5,19,-13.5,107,4.197887
7706,GSW @ BOS,-3.5,-16,12.5,100,-1.701408
7707,GSW @ BOS,-4.0,10,-14.0,107,3.032394
7708,GSW vs. BOS,4.0,10,-6.0,104,-2.645070


In [52]:
def add_percentage_features(df, span):
    """Add the following features for both team and opp:
    OREB_PCT, DREB_PCT, REB_PCT, TS_PCT, EFG_PCT, AST_RATIO, TOV_PCT, PIE.
    """
    
    df = df.copy()
    
    df['OREB_PCT_L{}'.format(span)] = df['OREB_L{}'.format(span)] / (df['OREB_L{}'.format(span)] + df['DREB_opp_L{}'.format(span)])
    df['OREB_PCT_opp_L{}'.format(span)] = df['OREB_opp_L{}'.format(span)] / (df['OREB_opp_L{}'.format(span)] + df['DREB_L{}'.format(span)])

    df['DREB_PCT_L{}'.format(span)] = df['DREB_L{}'.format(span)] / (df['DREB_L{}'.format(span)] + df['OREB_opp_L{}'.format(span)])
    df['DREB_PCT_opp_L{}'.format(span)] = df['DREB_opp_L{}'.format(span)] / (df['DREB_opp_L{}'.format(span)] + df['OREB_L{}'.format(span)])

    df['REB_PCT_L{}'.format(span)] = df['REB_L{}'.format(span)] / (df['REB_L{}'.format(span)] + df['REB_opp_L{}'.format(span)])
    df['REB_PCT_opp_L{}'.format(span)] = df['REB_opp_L{}'.format(span)] / (df['REB_opp_L{}'.format(span)] + df['REB_L{}'.format(span)])

    df['TS_PCT_L{}'.format(span)] = df['PTS_L{}'.format(span)] / ((2*(df['FG2A_L{}'.format(span)] + df['FG3A_L{}'.format(span)]) + 0.44*df['FTA_L{}'.format(span)]))
    
    df['TS_PCT_opp_L{}'.format(span)] = df['PTS_opp_L{}'.format(span)] / ((2*(df['FG2A_opp_L{}'.format(span)] + df['FG3A_opp_L{}'.format(span)]) + 0.44*df['FTA_opp_L{}'.format(span)]))

    df['EFG_PCT_L{}'.format(span)] = (df['FG2M_L{}'.format(span)] + 1.5*df['FG3M_L{}'.format(span)]) / (df['FG2A_L{}'.format(span)]
                                                                    + df['FG3A_L{}'.format(span)])
    df['EFG_PCT_opp_L{}'.format(span)] = (df['FG2M_opp_L{}'.format(span)] + 1.5*df['FG3M_opp_L{}'.format(span)]) / (df['FG2A_opp_L{}'.format(span)] 
                                                                 + df['FG3A_opp_L{}'.format(span)])

    df['AST_RATIO_L{}'.format(span)] = (df['AST_L{}'.format(span)] * 100) / df['PACE_L{}'.format(span)]
    df['AST_RATIO_opp_L{}'.format(span)] = (df['AST_opp_L{}'.format(span)] * 100) / df['PACE_opp_L{}'.format(span)]

    df['TOV_PCT_L{}'.format(span)] = 100*df['TOV_L{}'.format(span)] / (df['FG2A_L{}'.format(span)] 
                                               + df['FG3A_L{}'.format(span)] 
                                               + 0.44*df['FTA_L{}'.format(span)] 
                                               + df['TOV_L{}'.format(span)])
    
    df['TOV_PCT_opp_L{}'.format(span)] = 100*df['TOV_opp_L{}'.format(span)] / (df['FG2A_opp_L{}'.format(span)] 
                                             + df['FG3A_opp_L{}'.format(span)] 
                                             + 0.44*df['FTA_opp_L{}'.format(span)] 
                                             + df['TOV_opp_L{}'.format(span)])
    
    
    df['PIE_L{}'.format(span)] = ((df['PTS_L{}'.format(span)] + df['FG2M_L{}'.format(span)] + df['FG3M_L{}'.format(span)] + df['FTM_L{}'.format(span)] 
                 - df['FG2A_L{}'.format(span)] - df['FG3A_L{}'.format(span)] - df['FTA_L{}'.format(span)] 
                 + df['DREB_L{}'.format(span)] + df['OREB_L{}'.format(span)]/2
                + df['AST_L{}'.format(span)] + df['STL_L{}'.format(span)] + df['BLK_L{}'.format(span)]/2
                - df['PF_L{}'.format(span)] - df['TOV_L{}'.format(span)]) 
                 / (df['PTS_L{}'.format(span)] + df['PTS_opp_L{}'.format(span)] + df['FG2M_L{}'.format(span)] + df['FG2M_opp_L{}'.format(span)]
                   + df['FG3M_L{}'.format(span)] + df['FG3M_opp_L{}'.format(span)] + df['FTM_L{}'.format(span)] + df['FTM_opp_L{}'.format(span)]
                   - df['FG2A_L{}'.format(span)] - df['FG2A_opp_L{}'.format(span)] - df['FG3A_L{}'.format(span)] - df['FG3A_opp_L{}'.format(span)] 
                    - df['FTA_L{}'.format(span)] - df['FTA_opp_L{}'.format(span)] + df['DREB_L{}'.format(span)] + df['DREB_opp_L{}'.format(span)]
                    + (df['OREB_L{}'.format(span)]+df['OREB_opp_L{}'.format(span)])/2 + df['AST_L{}'.format(span)] + df['AST_opp_L{}'.format(span)]
                    + df['STL_L{}'.format(span)] + df['STL_opp_L{}'.format(span)] + (df['BLK_L{}'.format(span)] + df['BLK_opp_L{}'.format(span)])/2
                    - df['PF_L{}'.format(span)] - df['PF_opp_L{}'.format(span)] - df['TOV_L{}'.format(span)] - df['TOV_opp_L{}'.format(span)]))
        
    return df



In [53]:
add_percentage_features(team_stats_ewa_5, span=5)

,SEASON,TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_GAME,TEAM_SCORE,ML,SPREAD,ATS_DIFF,TEAM_COVERED,POINT_DIFF,WL,FG2M_L5,FG2A_L5,FG3M_L5,FG3A_L5,FTM_L5,FTA_L5,OREB_L5,DREB_L5,REB_L5,AST_L5,STL_L5,BLK_L5,TOV_L5,PF_L5,PTS_L5,PLUS_MINUS_L5,OFF_RATING_L5,DEF_RATING_L5,NET_RATING_L5,PACE_L5,POSS_L5,DIST_L5,ORBC_L5,DRBC_L5,RBC_L5,TCHS_L5,SAST_L5,FTAST_L5,PASS_L5,CFGM_L5,CFGA_L5,UFGM_L5,UFGA_L5,DFGM_L5,DFGA_L5,PTS_2PT_MR_L5,PTS_FB_L5,PTS_OFF_TOV_L5,PTS_PAINT_L5,AST_2PM_L5,AST_3PM_L5,UAST_2PM_L5,UAST_3PM_L5,FG2M_opp_L5,FG2A_opp_L5,FG3M_opp_L5,FG3A_opp_L5,FTM_opp_L5,FTA_opp_L5,OREB_opp_L5,DREB_opp_L5,REB_opp_L5,AST_opp_L5,STL_opp_L5,BLK_opp_L5,TOV_opp_L5,PF_opp_L5,PTS_opp_L5,PLUS_MINUS_opp_L5,OFF_RATING_opp_L5,DEF_RATING_opp_L5,NET_RATING_opp_L5,PACE_opp_L5,POSS_opp_L5,DIST_opp_L5,ORBC_opp_L5,DRBC_opp_L5,RBC_opp_L5,TCHS_opp_L5,SAST_opp_L5,FTAST_opp_L5,PASS_opp_L5,CFGM_opp_L5,CFGA_opp_L5,UFGM_opp_L5,UFGA_opp_L5,DFGM_opp_L5,DFGA_opp_L5,PTS_2PT_MR_opp_L5,PTS_FB_opp_L5,PTS_OFF_TOV_opp_L5,PTS_PAINT_opp_L5,AST_2PM_opp_L5,AST_3PM_opp_L5,UAST_2PM_opp_L5,UAST_3PM_opp_L5,AVG_ATS_DIFF_L5,WIN_PCT_L5,COVER_PCT_L5,OREB_PCT_L5,OREB_PCT_opp_L5,DREB_PCT_L5,DREB_PCT_opp_L5,REB_PCT_L5,REB_PCT_opp_L5,TS_PCT_L5,TS_PCT_opp_L5,EFG_PCT_L5,EFG_PCT_opp_L5,AST_RATIO_L5,AST_RATIO_opp_L5,TOV_PCT_L5,TOV_PCT_opp_L5,PIE_L5
0,2013-14,ATL,2013-10-30,0021300012,ATL @ DAL,0,109,1.408163,-6.0,3.0,1,-9,0,28.019324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-14,ATL,2013-11-01,0021300023,ATL vs. TOR,1,102,2.400000,3.5,-3.5,0,7,1,27.628782,50.241546,7.729469,23.188406,26.086957,33.816425,4.830918,27.053140,31.884058,26.086957,15.458937,4.830918,16.425121,19.323671,105.314010,-9.000000,104.800000,114.600000,-9.800000,103.500000,104.000000,17.760000,21.256039,44.444444,63.768116,384.541063,2.898551,2.898551,278.260870,16.425121,32.850242,19.323671,40.579710,9.661836,13.526570,15.458937,18.357488,25.120773,39.613527,19.323671,6.763285,7.729469,0.966184,31.884058,51.207729,10.628019,23.188406,18.357488,23.188406,6.763285,33.816425,40.579710,29.951691,6.763285,1.932367,19.323671,21.256039,114.009662,9.000000,114.600000,104.800000,9.800000,103.500000,103.000000,17.770000,24.154589,55.072464,74.396135,400.000000,2.898551,2.898551,285.990338,13.526570,26.086957,28.985507,48.309179,13.526570,18.357488,26.086957,17.391304,9.661836,35.748792,18.357488,10.628019,12.560386,0.000000,3.000000,NaN,NaN,0.125000,0.200000,0.800000,0.875000,0.440000,0.560000,0.651135,0.717064,0.534155,0.642857,25.204789,28.938832,15.682657,18.594273,0.465515
2,2013-14,ATL,2013-11-03,0021300046,ATL @ LAL,0,103,1.869565,1.0,3.0,1,-2,0,28.460147,54.201882,9.407577,23.801678,23.066362,33.105517,6.353420,29.768624,36.122044,25.592677,10.604628,3.827104,14.780575,15.308416,106.546656,0.600000,106.360000,105.840000,0.520000,98.400000,98.600000,17.064000,25.555047,54.409357,77.296720,390.658530,2.422578,1.790999,275.514874,17.938469,34.613781,19.097890,43.389779,13.338419,19.936944,13.762522,11.764048,17.627257,40.476990,15.939995,9.021103,10.670735,0.386473,33.595728,61.535723,8.672260,23.801678,12.395627,18.117468,9.021103,33.737096,42.758200,21.454361,5.863209,4.562421,15.308416,24.291889,105.603865,-0.600000,105.840000,106.360000,-0.520000,98.400000,98.200000,17.398000,31.767099,59.292143,86.600559,408.842105,3.054157,1.159420,291.238241,18.673786,41.382151,23.594203,43.955250,14.252733,18.711416,21.171625,11.377574,13.338419,43.983727,13.027206,7.409102,19.550470,0.631579,-0.900000,NaN,NaN,0.158477,0.232564,0.767436,0.841523,0.457935,0.542065,0.624638,0.591133,0.545764,0.546116,26.008818,21.803212,13.768512,14.093876,0.533663
3,2013-14,ATL,2

In [54]:
def build_league_avg_stats_df(df):
    """This function finds the league average for each statistic per season up to (and NOT including) the given date.
    """
    df = df.copy()
    
    df = df.drop(columns=['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'MATCHUP', 'HOME_GAME', 'TEAM_SCORE', 'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED',
       'POINT_DIFF', 'WL', 'RECORD'])

    grouped_df = df.groupby(['SEASON', 'GAME_DATE']).agg(['sum', 'count']).reset_index()
    grouped_df_cum = grouped_df.groupby(['SEASON']).cumsum().reset_index(drop=True)

    cum_stats_per_date = {}
    cum_stats_per_date['SEASON'] = grouped_df['SEASON']
    cum_stats_per_date['GAME_DATE'] = grouped_df['GAME_DATE']
    
    for col in grouped_df_cum.columns.get_level_values(0):
        avg = grouped_df_cum[col]['sum'] / grouped_df_cum[col]['count']
        
        cum_stats_per_date[col] = avg
        
    cum_stats_per_date = pd.DataFrame(cum_stats_per_date).sort_values('GAME_DATE')
    cum_stats_per_date.iloc[:, 2:] = cum_stats_per_date.iloc[:, 2:].shift(1)
    
    cum_stats_per_date = cum_stats_per_date.add_suffix('_LA')
    
    cum_stats_per_date = cum_stats_per_date.rename(columns={'SEASON_LA':'SEASON', 'GAME_DATE_LA':'GAME_DATE'})
        
    return cum_stats_per_date
    

league_avg_stats = build_league_avg_stats_df(stats_per_100)
league_avg_stats

C:\Users\Jordan Nishimura\AppData\Local\Temp\ipykernel_38028\1974283373.py:11: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_df_cum = grouped_df.groupby(['SEASON']).cumsum().reset_index(drop=True)


,SEASON,GAME_DATE,FG2M_LA,FG2A_LA,FG3M_LA,FG3A_LA,FTM_LA,FTA_LA,OREB_LA,DREB_LA,REB_LA,AST_LA,STL_LA,BLK_LA,TOV_LA,PF_LA,PTS_LA,PLUS_MINUS_LA,OFF_RATING_LA,DEF_RATING_LA,NET_RATING_LA,PACE_LA,POSS_LA,DIST_LA,ORBC_LA,DRBC_LA,RBC_LA,TCHS_LA,SAST_LA,FTAST_LA,PASS_LA,CFGM_LA,CFGA_LA,UFGM_LA,UFGA_LA,DFGM_LA,DFGA_LA,PTS_2PT_MR_LA,PTS_FB_LA,PTS_OFF_TOV_LA,PTS_PAINT_LA,AST_2PM_LA,AST_3PM_LA,UAST_2PM_LA,UAST_3PM_LA
0,2013-14,2013-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-14,2013-10-30,29.033013,62.458149,9.605069,22.626634,16.978075,24.874896,11.523023,32.465709,43.988732,22.779638,9.250459,7.808619,19.613349,22.481729,103.859308,0.0,103.850000,103.850000,0.000000e+00,97.000000,97.000000,17.043333,40.153724,67.768298,103.474097,443.795226,2.046657,1.530943,316.969975,17.727796,39.130396,20.910285,45.954387,14.230298,23.343954,15.874644,13.376175,24.931598,41.168053,13.867588,8.041725,14.133996,1.039529
2,2013-14,2013-10-31,30.167767,63.028128,8.236339,21.489982,18.061842,24.644024,10.623191,32.467354,43.090545,21.846856,8.234577,5.869893,17.969809,22.848289,103.106393,0.0,102.458824,102.458824,0.000000e+00,96.846471,97.470588,16.982941,35.045442,64.273045,96.088620,432.125387,2.191121,1.946505,307.581564,17.092559,38.152311,21.138350,46.073859,14.360940,23.507155,18.001209,15.209501,19.164916,41.299717,14.458297,6.752665,14.831313,1.208885
3,2013-14,2013-11-01,29.954447,62.715390,8.236711,21.602585,18.327344,25.056147,10.904643,32.175387,43.080030,22.020270,8.405928,5.721213,18.154547,23.211420,102.946371,0.0,102.368421,102.368421,0.000000e+00,97.020526,97.578947,16.881842,35.800073,63.813960,96.419730,427.625551,2.337315,1.869226,303.608940,17.071229,38.141307,20.906483,45.623061,14.293488,23.470549,17.770552,15.298669,19.117551,41.158805,14.669715,6.673929,14.391344,1.263085
4,2013-14,2013-11-02,29.915777,63.868616,8.002773,21.521141,19.393497,26.659643,11.579612,32.619413,44.199026,21.587076,8.313287,5.526029,17.276477,23.690646,103.233371,0.0,102.539062,102.539062,0.000000e+00,96.447188,97.078125,16.957031,37.102309,65.695297,99.572065,434.508240,2.562918,1.830899,309.014177,17.561363,39.602342,20.230454,45.458711,14.105067,23.303792,16.995895,15.017298,18.323367,41.777088,14.400417,6.486898,14.672128,1.139966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835,2021-22,2022-06-05,28.464632,53.486092,12.611523,35.668407,17.231911,22.223201,10.444859,34.514375,44.959234,24.893718,7.685570,4.768411,13.934763,20.057479,111.995745,0.0,111.400076,111.400076,1.417396e-18,98.578399,99.104802,17.796833,24.692302,55.634655,78.024629,411.201445,3.291638,2.428174,288.410007,17.238369,34.413404,23.670494,54.350350,16.884367,25.975233,8.776408,12.122283,15.983176,47.194669,14.175536,9.959853,13.495272,2.033486
1836,2021-22,2022-06-08,28.458185,53.481289,12.613636,35.670143,17.229333,22.220727,10.441591,34.516600,44.958191,24.893897,7.687528,4.768294,13.936258,20.055873,111.986609,0.0,111.391432,111.391432,1.416317e-18,98.577578,99.103580,17.796561,24.689118,55.639152,78.027305,411.199163,3.292646,2.428278,288.405945,17.236176,34.414925,23.668480,54.346060,16.882444,25.975760,8.774801,12.122424,15.988969,47.183721,14.175676,9.960470,13.488900,2.035452
1837,2021-22,2022-06-10,28.457063,53.477633,12.615311,35.673197,17.228301,22.219520,10.442100,34.513284,44.955384,24.895085,7.687315,4.769497,13.937329,20.056716,111.988359,0.0,111.393607,111.393607,1.415239e-18,98.574475,99.100076,17.795708,24.687241,55.630632,78.017452,411.205136,3.292959,2.429651,288.410339,17.236347,34.416518,23.668990,54.344162,16.882078,25.976125,8.777787,12.121654,15.991297,47.178818,14.176163,9.960943,13.487090,2.036319
1838,2021-22,2022-06-13,28.452850,53.474592,12.617599,35.678151,17.225497,22.2160

In [55]:
test = pd.merge(team_stats_ewa_5, clean_df, on=['TEAM_ABBREVIATION', 'GAME_ID'])

test['PTS_100'] = 100.0* test['PTS'] / test['PACE']

test.loc[test['TEAM_ABBREVIATION'] == 'SAS', ['TEAM_ABBREVIATION', 'GAME_DATE_x', 'WL_x', 'WL_y', 'WIN_PCT_L5', 'PTS_L5', 'PTS', 'PTS_100']].head(20)

,TEAM_ABBREVIATION,GAME_DATE_x,WL_x,WL_y,WIN_PCT_L5,PTS_L5,PTS,PTS_100
19879,SAS,2013-10-30,1,1,NaN,NaN,101,106.315789
19880,SAS,2013-11-01,1,1,NaN,106.315789,91,95.789474
19881,SAS,2013-11-02,0,0,NaN,100.000000,105,107.692308
19882,SAS,2013-11-05,1,1,NaN,103.643725,102,98.550725
19883,SAS,2013-11-06,1,1,NaN,101.528171,99,100.507614
19884,SAS,2013-11-08,1,1,0.8,101.136393,76,85.875706
19885,SAS,2013-11-10,1,1,0.8,95.559932,120,126.315789
19886,SAS,2013-11-11,1,1,0.8,106.449208,109,117.204301
19887,SAS,2013-11-13,1,1,1.0,110.179801,92,97.354497
19888,SAS,2013-11-15,1,1,1.0,105.790525,91,94.791667


In [56]:
spurs_pts = test.loc[test['TEAM_ABBREVIATION']=='SAS', ['TEAM_ABBREVIATION', 'PTS']]

spurs_pts.groupby(['TEAM_ABBREVIATION'])['PTS'].ewm(span=5).mean().reset_index(drop=True)

0      101.000000
1       95.000000
2       99.736842
3      100.676923
4      100.033175
          ...    
780    117.713848
781    117.142565
782    118.428377
783    110.285584
784    113.523723
Name: PTS, Length: 785, dtype: float64

In [57]:
team_stats_ewa_5 = build_team_avg_stats_df(matchups, span=5)
team_stats_ewa_5 = add_percentage_features(team_stats_ewa_5, span=5)

team_stats_ewa_10 = build_team_avg_stats_df(matchups, span=10)
team_stats_ewa_10 = add_percentage_features(team_stats_ewa_10, span=10)

team_stats_ewa_20 = build_team_avg_stats_df(matchups, span=20)
team_stats_ewa_20 = add_percentage_features(team_stats_ewa_20, span=20)


temp = pd.merge(team_stats_ewa_5, team_stats_ewa_10, how='inner', on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                          'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                          'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                          'POINT_DIFF', 'WL'])

team_stats_full = pd.merge(temp, team_stats_ewa_20, how='inner', on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                          'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                          'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                          'POINT_DIFF', 'WL'])

team_stats_full = team_stats_full.sort_values(['GAME_DATE', 'GAME_ID', 'HOME_GAME']).reset_index(drop=True)
team_stats_full


,SEASON,TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_GAME,TEAM_SCORE,ML,SPREAD,ATS_DIFF,TEAM_COVERED,POINT_DIFF,WL,FG2M_L5,FG2A_L5,FG3M_L5,FG3A_L5,FTM_L5,FTA_L5,OREB_L5,DREB_L5,REB_L5,AST_L5,STL_L5,BLK_L5,TOV_L5,PF_L5,PTS_L5,PLUS_MINUS_L5,OFF_RATING_L5,DEF_RATING_L5,NET_RATING_L5,PACE_L5,POSS_L5,DIST_L5,ORBC_L5,DRBC_L5,RBC_L5,TCHS_L5,SAST_L5,FTAST_L5,PASS_L5,CFGM_L5,CFGA_L5,UFGM_L5,UFGA_L5,DFGM_L5,DFGA_L5,PTS_2PT_MR_L5,PTS_FB_L5,PTS_OFF_TOV_L5,PTS_PAINT_L5,AST_2PM_L5,AST_3PM_L5,UAST_2PM_L5,UAST_3PM_L5,FG2M_opp_L5,FG2A_opp_L5,FG3M_opp_L5,FG3A_opp_L5,FTM_opp_L5,FTA_opp_L5,OREB_opp_L5,DREB_opp_L5,REB_opp_L5,AST_opp_L5,STL_opp_L5,BLK_opp_L5,TOV_opp_L5,PF_opp_L5,PTS_opp_L5,PLUS_MINUS_opp_L5,OFF_RATING_opp_L5,DEF_RATING_opp_L5,NET_RATING_opp_L5,PACE_opp_L5,POSS_opp_L5,DIST_opp_L5,ORBC_opp_L5,DRBC_opp_L5,RBC_opp_L5,TCHS_opp_L5,SAST_opp_L5,FTAST_opp_L5,PASS_opp_L5,CFGM_opp_L5,CFGA_opp_L5,UFGM_opp_L5,UFGA_opp_L5,DFGM_opp_L5,DFGA_opp_L5,PTS_2PT_MR_opp_L5,PTS_FB_opp_L5,PTS_OFF_TOV_opp_L5,PTS_PAINT_opp_L5,AST_2PM_opp_L5,AST_3PM_opp_L5,UAST_2PM_opp_L5,UAST_3PM_opp_L5,AVG_ATS_DIFF_L5,WIN_PCT_L5,COVER_PCT_L5,OREB_PCT_L5,OREB_PCT_opp_L5,DREB_PCT_L5,DREB_PCT_opp_L5,REB_PCT_L5,REB_PCT_opp_L5,TS_PCT_L5,TS_PCT_opp_L5,EFG_PCT_L5,EFG_PCT_opp_L5,AST_RATIO_L5,AST_RATIO_opp_L5,TOV_PCT_L5,...,REB_PCT_L10,REB_PCT_opp_L10,TS_PCT_L10,TS_PCT_opp_L10,EFG_PCT_L10,EFG_PCT_opp_L10,AST_RATIO_L10,AST_RATIO_opp_L10,TOV_PCT_L10,TOV_PCT_opp_L10,PIE_L10,FG2M_L20,FG2A_L20,FG3M_L20,FG3A_L20,FTM_L20,FTA_L20,OREB_L20,DREB_L20,REB_L20,AST_L20,STL_L20,BLK_L20,TOV_L20,PF_L20,PTS_L20,PLUS_MINUS_L20,OFF_RATING_L20,DEF_RATING_L20,NET_RATING_L20,PACE_L20,POSS_L20,DIST_L20,ORBC_L20,DRBC_L20,RBC_L20,TCHS_L20,SAST_L20,FTAST_L20,PASS_L20,CFGM_L20,CFGA_L20,UFGM_L20,UFGA_L20,DFGM_L20,DFGA_L20,PTS_2PT_MR_L20,PTS_FB_L20,PTS_OFF_TOV_L20,PTS_PAINT_L20,AST_2PM_L20,AST_3PM_L20,UAST_2PM_L20,UAST_3PM_L20,FG2M_opp_L20,FG2A_opp_L20,FG3M_opp_L20,FG3A_opp_L20,FTM_opp_L20,FTA_opp_L20,OREB_opp_L20,DREB_opp_L20,REB_opp_L20,AST_opp_L20,STL_opp_L20,BLK_opp_L20,TOV_opp_L20,PF_opp_L20,PTS_opp_L20,PLUS_MINUS_opp_L20,OFF_RATING_opp_L20,DEF_RATING_opp_L20,NET_RATING_opp_L20,PACE_opp_L20,POSS_opp_L20,DIST_opp_L20,ORBC_opp_L20,DRBC_opp_L20,RBC_opp_L20,TCHS_opp_L20,SAST_opp_L20,FTAST_opp_L20,PASS_opp_L20,CFGM_opp_L20,CFGA_opp_L20,UFGM_opp_L20,UFGA_opp_L20,DFGM_opp_L20,DFGA_opp_L20,PTS_2PT_MR_opp_L20,PTS_FB_opp_L20,PTS_OFF_TOV_opp_L20,PTS_PAINT_opp_L20,AST_2PM_opp_L20,AST_3PM_opp_L20,UAST_2PM_opp_L20,UAST_3PM_opp_L20,AVG_ATS_DIFF_L20,WIN_PCT_L20,COVER_PCT_L20,OREB_PCT_L20,OREB_PCT_opp_L20,DREB_PCT_L20,DREB_PCT_opp_L20,REB_PCT_L20,REB_PCT_opp_L20,TS_PCT_L20,TS_PCT_opp_L20,EFG_PCT_L20,EFG_PCT_opp_L20,AST_RATIO_L20,AST_RATIO_opp_L20,TOV_PCT_L20,TOV_PCT_opp_L20,PIE_L20
0,2013-14,ORL,2013-10-29,0021300001,ORL @ IND,0,87,1.090498,-12.5,-2.5,0,-10,0,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-14,IND,2013-10-29,0021300001,IND vs. ORL,1,97,8.600000,12.5,2.5,1,10,1,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [58]:
def add_rest_days(df):
    
    df['prev_game'] = df.groupby(['SEASON', 'TEAM_ABBREVIATION'])['GAME_DATE'].shift(1)

    df['REST'] = (df['GAME_DATE'] - df['prev_game']) / np.timedelta64(1, 'D')
            
    df.loc[df['REST'] >= 8, 'REST'] = 8
    
    df = df.drop(columns=['prev_game'])
    
    return df

team_stats_full = add_rest_days(team_stats_full)


In [73]:
clean_spreads

,SEASON,GM_DATE,HOME_TEAM,AWAY_TEAM,HIGHEST_HOME_SPREAD,HIGHEST_AWAY_SPREAD
0,2006-07,2006-10-31,MIA,CHI,-4.5,4.5
1,2006-07,2006-10-31,LAL,PHX,8.5,-8.5
2,2006-07,2006-11-02,DAL,SAS,-3.0,3.0
3,2006-07,2006-11-02,LAC,DEN,-4.5,4.5
4,2006-07,2006-11-03,MEM,CHA,-5.0,5.0
...,...,...,...,...,...,...
20197,2021-22,2022-06-05,GSW,BOS,-5.5,5.0
20198,2021-22,2022-06-08,BOS,GSW,-3.5,3.5
20199,2021-22,2022-06-10,BOS,GSW,-4.0,4.0
20200,2021-22,2022-06-13,GSW,BOS,-4.0,4.0


In [81]:
matchups[['TEAM_ABBREVIATION', 'GAME_DATE', 'HOME_GAME', 'MATCHUP', 'SPREAD']]

,TEAM_ABBREVIATION,GAME_DATE,HOME_GAME,MATCHUP,SPREAD
1,LAC,2013-10-29,0,LAC @ LAL,10.0
2,LAL,2013-10-29,1,LAL vs. LAC,-10.0
5,CHI,2013-10-29,0,CHI @ MIA,-5.0
6,MIA,2013-10-29,1,MIA vs. CHI,5.0
9,IND,2013-10-29,1,IND vs. ORL,12.5
...,...,...,...,...,...
46002,GSW,2022-06-10,0,GSW @ BOS,-4.0
46005,BOS,2022-06-13,0,BOS @ GSW,-4.0
46006,GSW,2022-06-13,1,GSW vs. BOS,4.0
46009,BOS,2022-06-16,1,BOS vs. GSW,4.0


In [71]:

team_stats_ewa_5.sort_values(['GAME_DATE', 'GAME_ID', 'HOME_GAME'])[['TEAM_ABBREVIATION', 'MATCHUP', 'HOME_GAME', 'SPREAD']]

,TEAM_ABBREVIATION,MATCHUP,HOME_GAME,SPREAD
16127,ORL,ORL @ IND,0,-12.5
8507,IND,IND vs. ORL,1,12.5
3078,CHI,CHI @ MIA,0,-5.0
11575,MIA,MIA vs. CHI,1,5.0
9271,LAC,LAC @ LAL,0,10.0
...,...,...,...,...
2353,BOS,BOS vs. GSW,1,4.0
2354,BOS,BOS @ GSW,0,-4.0
7708,GSW,GSW vs. BOS,1,4.0
7709,GSW,GSW @ BOS,0,-4.0


In [59]:
team_stats_full

,SEASON,TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_GAME,TEAM_SCORE,ML,SPREAD,ATS_DIFF,TEAM_COVERED,POINT_DIFF,WL,FG2M_L5,FG2A_L5,FG3M_L5,FG3A_L5,FTM_L5,FTA_L5,OREB_L5,DREB_L5,REB_L5,AST_L5,STL_L5,BLK_L5,TOV_L5,PF_L5,PTS_L5,PLUS_MINUS_L5,OFF_RATING_L5,DEF_RATING_L5,NET_RATING_L5,PACE_L5,POSS_L5,DIST_L5,ORBC_L5,DRBC_L5,RBC_L5,TCHS_L5,SAST_L5,FTAST_L5,PASS_L5,CFGM_L5,CFGA_L5,UFGM_L5,UFGA_L5,DFGM_L5,DFGA_L5,PTS_2PT_MR_L5,PTS_FB_L5,PTS_OFF_TOV_L5,PTS_PAINT_L5,AST_2PM_L5,AST_3PM_L5,UAST_2PM_L5,UAST_3PM_L5,FG2M_opp_L5,FG2A_opp_L5,FG3M_opp_L5,FG3A_opp_L5,FTM_opp_L5,FTA_opp_L5,OREB_opp_L5,DREB_opp_L5,REB_opp_L5,AST_opp_L5,STL_opp_L5,BLK_opp_L5,TOV_opp_L5,PF_opp_L5,PTS_opp_L5,PLUS_MINUS_opp_L5,OFF_RATING_opp_L5,DEF_RATING_opp_L5,NET_RATING_opp_L5,PACE_opp_L5,POSS_opp_L5,DIST_opp_L5,ORBC_opp_L5,DRBC_opp_L5,RBC_opp_L5,TCHS_opp_L5,SAST_opp_L5,FTAST_opp_L5,PASS_opp_L5,CFGM_opp_L5,CFGA_opp_L5,UFGM_opp_L5,UFGA_opp_L5,DFGM_opp_L5,DFGA_opp_L5,PTS_2PT_MR_opp_L5,PTS_FB_opp_L5,PTS_OFF_TOV_opp_L5,PTS_PAINT_opp_L5,AST_2PM_opp_L5,AST_3PM_opp_L5,UAST_2PM_opp_L5,UAST_3PM_opp_L5,AVG_ATS_DIFF_L5,WIN_PCT_L5,COVER_PCT_L5,OREB_PCT_L5,OREB_PCT_opp_L5,DREB_PCT_L5,DREB_PCT_opp_L5,REB_PCT_L5,REB_PCT_opp_L5,TS_PCT_L5,TS_PCT_opp_L5,EFG_PCT_L5,EFG_PCT_opp_L5,AST_RATIO_L5,AST_RATIO_opp_L5,TOV_PCT_L5,...,REB_PCT_opp_L10,TS_PCT_L10,TS_PCT_opp_L10,EFG_PCT_L10,EFG_PCT_opp_L10,AST_RATIO_L10,AST_RATIO_opp_L10,TOV_PCT_L10,TOV_PCT_opp_L10,PIE_L10,FG2M_L20,FG2A_L20,FG3M_L20,FG3A_L20,FTM_L20,FTA_L20,OREB_L20,DREB_L20,REB_L20,AST_L20,STL_L20,BLK_L20,TOV_L20,PF_L20,PTS_L20,PLUS_MINUS_L20,OFF_RATING_L20,DEF_RATING_L20,NET_RATING_L20,PACE_L20,POSS_L20,DIST_L20,ORBC_L20,DRBC_L20,RBC_L20,TCHS_L20,SAST_L20,FTAST_L20,PASS_L20,CFGM_L20,CFGA_L20,UFGM_L20,UFGA_L20,DFGM_L20,DFGA_L20,PTS_2PT_MR_L20,PTS_FB_L20,PTS_OFF_TOV_L20,PTS_PAINT_L20,AST_2PM_L20,AST_3PM_L20,UAST_2PM_L20,UAST_3PM_L20,FG2M_opp_L20,FG2A_opp_L20,FG3M_opp_L20,FG3A_opp_L20,FTM_opp_L20,FTA_opp_L20,OREB_opp_L20,DREB_opp_L20,REB_opp_L20,AST_opp_L20,STL_opp_L20,BLK_opp_L20,TOV_opp_L20,PF_opp_L20,PTS_opp_L20,PLUS_MINUS_opp_L20,OFF_RATING_opp_L20,DEF_RATING_opp_L20,NET_RATING_opp_L20,PACE_opp_L20,POSS_opp_L20,DIST_opp_L20,ORBC_opp_L20,DRBC_opp_L20,RBC_opp_L20,TCHS_opp_L20,SAST_opp_L20,FTAST_opp_L20,PASS_opp_L20,CFGM_opp_L20,CFGA_opp_L20,UFGM_opp_L20,UFGA_opp_L20,DFGM_opp_L20,DFGA_opp_L20,PTS_2PT_MR_opp_L20,PTS_FB_opp_L20,PTS_OFF_TOV_opp_L20,PTS_PAINT_opp_L20,AST_2PM_opp_L20,AST_3PM_opp_L20,UAST_2PM_opp_L20,UAST_3PM_opp_L20,AVG_ATS_DIFF_L20,WIN_PCT_L20,COVER_PCT_L20,OREB_PCT_L20,OREB_PCT_opp_L20,DREB_PCT_L20,DREB_PCT_opp_L20,REB_PCT_L20,REB_PCT_opp_L20,TS_PCT_L20,TS_PCT_opp_L20,EFG_PCT_L20,EFG_PCT_opp_L20,AST_RATIO_L20,AST_RATIO_opp_L20,TOV_PCT_L20,TOV_PCT_opp_L20,PIE_L20,REST
0,2013-14,ORL,2013-10-29,0021300001,ORL @ IND,0,87,1.090498,-12.5,-2.5,0,-10,0,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-14,IND,2013-10-29,0021300001,IND vs. ORL,1,97,8.600000,12.5,2.5,1,10,1,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [60]:
def make_matchups_2(df):
    df = df.copy()
    
    home_teams = df.loc[df['HOME_GAME'] == 1]
    home_teams = home_teams.add_prefix('HOME_')

    away_teams = df.loc[df['HOME_GAME'] == 0]
    away_teams = away_teams.add_prefix('AWAY_')


    away_teams = away_teams.drop(columns=['AWAY_SEASON', 'AWAY_TEAM_ABBREVIATION', 'AWAY_GAME_DATE',
                            'AWAY_MATCHUP', 'AWAY_HOME_GAME', 'AWAY_POINT_DIFF', 'AWAY_WL',
                            'AWAY_ATS_DIFF', 'AWAY_SPREAD', 'AWAY_TEAM_COVERED', 'AWAY_TEAM_SCORE'])

    full_matchup_ewa = pd.merge(home_teams, away_teams, how = 'inner',
                                left_on = 'HOME_GAME_ID', right_on = 'AWAY_GAME_ID') 
    
    full_matchup_ewa = full_matchup_ewa.rename(columns = {'HOME_SEASON':'SEASON',
                                                          'HOME_GAME_DATE':'GAME_DATE',
                                                          'HOME_GAME_ID':'GAME_ID',
                                                          'HOME_MATCHUP':'MATCHUP'})
    
    full_matchup_ewa = full_matchup_ewa.drop(columns=['AWAY_GAME_ID'])
    
    return full_matchup_ewa




In [61]:
team_stats_full_matchups = make_matchups_2(team_stats_full)

team_stats_full_matchups

,SEASON,HOME_TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_HOME_GAME,HOME_TEAM_SCORE,HOME_ML,HOME_SPREAD,HOME_ATS_DIFF,HOME_TEAM_COVERED,HOME_POINT_DIFF,HOME_WL,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_REB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_PTS_L5,HOME_PLUS_MINUS_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_NET_RATING_L5,HOME_PACE_L5,HOME_POSS_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_REB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_PTS_opp_L5,HOME_PLUS_MINUS_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_NET_RATING_opp_L5,HOME_PACE_opp_L5,HOME_POSS_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_AVG_ATS_DIFF_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,...,AWAY_REB_PCT_opp_L10,AWAY_TS_PCT_L10,AWAY_TS_PCT_opp_L10,AWAY_EFG_PCT_L10,AWAY_EFG_PCT_opp_L10,AWAY_AST_RATIO_L10,AWAY_AST_RATIO_opp_L10,AWAY_TOV_PCT_L10,AWAY_TOV_PCT_opp_L10,AWAY_PIE_L10,AWAY_FG2M_L20,AWAY_FG2A_L20,AWAY_FG3M_L20,AWAY_FG3A_L20,AWAY_FTM_L20,AWAY_FTA_L20,AWAY_OREB_L20,AWAY_DREB_L20,AWAY_REB_L20,AWAY_AST_L20,AWAY_STL_L20,AWAY_BLK_L20,AWAY_TOV_L20,AWAY_PF_L20,AWAY_PTS_L20,AWAY_PLUS_MINUS_L20,AWAY_OFF_RATING_L20,AWAY_DEF_RATING_L20,AWAY_NET_RATING_L20,AWAY_PACE_L20,AWAY_POSS_L20,AWAY_DIST_L20,AWAY_ORBC_L20,AWAY_DRBC_L20,AWAY_RBC_L20,AWAY_TCHS_L20,AWAY_SAST_L20,AWAY_FTAST_L20,AWAY_PASS_L20,AWAY_CFGM_L20,AWAY_CFGA_L20,AWAY_UFGM_L20,AWAY_UFGA_L20,AWAY_DFGM_L20,AWAY_DFGA_L20,AWAY_PTS_2PT_MR_L20,AWAY_PTS_FB_L20,AWAY_PTS_OFF_TOV_L20,AWAY_PTS_PAINT_L20,AWAY_AST_2PM_L20,AWAY_AST_3PM_L20,AWAY_UAST_2PM_L20,AWAY_UAST_3PM_L20,AWAY_FG2M_opp_L20,AWAY_FG2A_opp_L20,AWAY_FG3M_opp_L20,AWAY_FG3A_opp_L20,AWAY_FTM_opp_L20,AWAY_FTA_opp_L20,AWAY_OREB_opp_L20,AWAY_DREB_opp_L20,AWAY_REB_opp_L20,AWAY_AST_opp_L20,AWAY_STL_opp_L20,AWAY_BLK_opp_L20,AWAY_TOV_opp_L20,AWAY_PF_opp_L20,AWAY_PTS_opp_L20,AWAY_PLUS_MINUS_opp_L20,AWAY_OFF_RATING_opp_L20,AWAY_DEF_RATING_opp_L20,AWAY_NET_RATING_opp_L20,AWAY_PACE_opp_L20,AWAY_POSS_opp_L20,AWAY_DIST_opp_L20,AWAY_ORBC_opp_L20,AWAY_DRBC_opp_L20,AWAY_RBC_opp_L20,AWAY_TCHS_opp_L20,AWAY_SAST_opp_L20,AWAY_FTAST_opp_L20,AWAY_PASS_opp_L20,AWAY_CFGM_opp_L20,AWAY_CFGA_opp_L20,AWAY_UFGM_opp_L20,AWAY_UFGA_opp_L20,AWAY_DFGM_opp_L20,AWAY_DFGA_opp_L20,AWAY_PTS_2PT_MR_opp_L20,AWAY_PTS_FB_opp_L20,AWAY_PTS_OFF_TOV_opp_L20,AWAY_PTS_PAINT_opp_L20,AWAY_AST_2PM_opp_L20,AWAY_AST_3PM_opp_L20,AWAY_UAST_2PM_opp_L20,AWAY_UAST_3PM_opp_L20,AWAY_AVG_ATS_DIFF_L20,AWAY_WIN_PCT_L20,AWAY_COVER_PCT_L20,AWAY_OREB_PCT_L20,AWAY_OREB_PCT_opp_L20,AWAY_DREB_PCT_L20,AWAY_DREB_PCT_opp_L20,AWAY_REB_PCT_L20,AWAY_REB_PCT_opp_L20,AWAY_TS_PCT_L20,AWAY_TS_PCT_opp_L20,AWAY_EFG_PCT_L20,AWAY_EFG_PCT_opp_L20,AWAY_AST_RATIO_L20,AWAY_AST_RATIO_opp_L20,AWAY_TOV_PCT_L20,AWAY_TOV_PCT_opp_L20,AWAY_PIE_L20,AWAY_REST
0,2013-14,IND,2013-10-29,0021300001,IND vs. ORL,1,97,8.600000,12.5,2.5,1,10,1,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [63]:
team_stats_full_matchups

,SEASON,HOME_TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_HOME_GAME,HOME_TEAM_SCORE,HOME_ML,HOME_SPREAD,HOME_ATS_DIFF,HOME_TEAM_COVERED,HOME_POINT_DIFF,HOME_WL,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_REB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_PTS_L5,HOME_PLUS_MINUS_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_NET_RATING_L5,HOME_PACE_L5,HOME_POSS_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_REB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_PTS_opp_L5,HOME_PLUS_MINUS_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_NET_RATING_opp_L5,HOME_PACE_opp_L5,HOME_POSS_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_AVG_ATS_DIFF_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,...,AWAY_REB_PCT_opp_L10,AWAY_TS_PCT_L10,AWAY_TS_PCT_opp_L10,AWAY_EFG_PCT_L10,AWAY_EFG_PCT_opp_L10,AWAY_AST_RATIO_L10,AWAY_AST_RATIO_opp_L10,AWAY_TOV_PCT_L10,AWAY_TOV_PCT_opp_L10,AWAY_PIE_L10,AWAY_FG2M_L20,AWAY_FG2A_L20,AWAY_FG3M_L20,AWAY_FG3A_L20,AWAY_FTM_L20,AWAY_FTA_L20,AWAY_OREB_L20,AWAY_DREB_L20,AWAY_REB_L20,AWAY_AST_L20,AWAY_STL_L20,AWAY_BLK_L20,AWAY_TOV_L20,AWAY_PF_L20,AWAY_PTS_L20,AWAY_PLUS_MINUS_L20,AWAY_OFF_RATING_L20,AWAY_DEF_RATING_L20,AWAY_NET_RATING_L20,AWAY_PACE_L20,AWAY_POSS_L20,AWAY_DIST_L20,AWAY_ORBC_L20,AWAY_DRBC_L20,AWAY_RBC_L20,AWAY_TCHS_L20,AWAY_SAST_L20,AWAY_FTAST_L20,AWAY_PASS_L20,AWAY_CFGM_L20,AWAY_CFGA_L20,AWAY_UFGM_L20,AWAY_UFGA_L20,AWAY_DFGM_L20,AWAY_DFGA_L20,AWAY_PTS_2PT_MR_L20,AWAY_PTS_FB_L20,AWAY_PTS_OFF_TOV_L20,AWAY_PTS_PAINT_L20,AWAY_AST_2PM_L20,AWAY_AST_3PM_L20,AWAY_UAST_2PM_L20,AWAY_UAST_3PM_L20,AWAY_FG2M_opp_L20,AWAY_FG2A_opp_L20,AWAY_FG3M_opp_L20,AWAY_FG3A_opp_L20,AWAY_FTM_opp_L20,AWAY_FTA_opp_L20,AWAY_OREB_opp_L20,AWAY_DREB_opp_L20,AWAY_REB_opp_L20,AWAY_AST_opp_L20,AWAY_STL_opp_L20,AWAY_BLK_opp_L20,AWAY_TOV_opp_L20,AWAY_PF_opp_L20,AWAY_PTS_opp_L20,AWAY_PLUS_MINUS_opp_L20,AWAY_OFF_RATING_opp_L20,AWAY_DEF_RATING_opp_L20,AWAY_NET_RATING_opp_L20,AWAY_PACE_opp_L20,AWAY_POSS_opp_L20,AWAY_DIST_opp_L20,AWAY_ORBC_opp_L20,AWAY_DRBC_opp_L20,AWAY_RBC_opp_L20,AWAY_TCHS_opp_L20,AWAY_SAST_opp_L20,AWAY_FTAST_opp_L20,AWAY_PASS_opp_L20,AWAY_CFGM_opp_L20,AWAY_CFGA_opp_L20,AWAY_UFGM_opp_L20,AWAY_UFGA_opp_L20,AWAY_DFGM_opp_L20,AWAY_DFGA_opp_L20,AWAY_PTS_2PT_MR_opp_L20,AWAY_PTS_FB_opp_L20,AWAY_PTS_OFF_TOV_opp_L20,AWAY_PTS_PAINT_opp_L20,AWAY_AST_2PM_opp_L20,AWAY_AST_3PM_opp_L20,AWAY_UAST_2PM_opp_L20,AWAY_UAST_3PM_opp_L20,AWAY_AVG_ATS_DIFF_L20,AWAY_WIN_PCT_L20,AWAY_COVER_PCT_L20,AWAY_OREB_PCT_L20,AWAY_OREB_PCT_opp_L20,AWAY_DREB_PCT_L20,AWAY_DREB_PCT_opp_L20,AWAY_REB_PCT_L20,AWAY_REB_PCT_opp_L20,AWAY_TS_PCT_L20,AWAY_TS_PCT_opp_L20,AWAY_EFG_PCT_L20,AWAY_EFG_PCT_opp_L20,AWAY_AST_RATIO_L20,AWAY_AST_RATIO_opp_L20,AWAY_TOV_PCT_L20,AWAY_TOV_PCT_opp_L20,AWAY_PIE_L20,AWAY_REST
0,2013-14,IND,2013-10-29,0021300001,IND vs. ORL,1,97,8.600000,12.5,2.5,1,10,1,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
connection = sqlite3.connect('../data/nba.db')

team_stats_full_matchups.to_sql('team_stats_ewa_matchup', connection, if_exists='replace')

connection.close()
